In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.morphology as morphology
import skimage.measure as measure

%matplotlib inline

In [ ]:
def read_img(img_num):
    path = #insert path to data
    img_name = str(img_num) + '.jpg'
    img = cv2.imread(path + img_name, cv2.IMREAD_GRAYSCALE)
    return img

In [ ]:
def find_graph_structure(img_num):
    img = read_img(img_num)
    # бинаризуем изображение по порогу
    _, threshold = cv2.threshold(img, 80, 1, type=cv2.THRESH_BINARY_INV)
    # удалим ошибочно определяемые как части графа затемнения по краям изображения
    threshold[:20, :] = 0
    threshold[-20:, :] = 0
    threshold[:, :20] = 0
    threshold[:, -20:] = 0
    # применим операцию замыкания, чтобы удалить небольшие промежутки в получившемся контуре графа                             
    convex_threshold = morphology.binary_closing(threshold, morphology.star(10))
    # построим скелет 
    skeleton = morphology.skeletonize(convex_threshold)
    skeleton = skeleton.astype(np.uint8)
    # сделаем контур скелета более широким, чтобы корректно работала операция поиска контуров на изображении
    convex_skeleton = morphology.dilation(skeleton, morphology.disk(3))
    cnts, _ = cv2.findContours(convex_skeleton.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    # удалим ошибочно найденные контуры маленького периметра
    for i in range(len(cnts) - 1, -1, -1):
        if cv2.arcLength(cnts[i], False) < 100:
            del cnts[i]
    if(len(cnts) == 6):
        print('Граф класса 3')
    elif(len(cnts) == 5):
        print('Граф класса 1')
    else:
        sum_img = np.zeros((img.shape[0], img.shape[1]))
        for ind in range(len(cnts)):
            blank_img = np.zeros((img.shape[0], img.shape[1]))
            cv2.drawContours(blank_img, cnts, ind, 1, 10)
            sum_img += blank_img
        _, inside_borders = cv2.threshold(sum_img, 1, 1, type=cv2.THRESH_BINARY)
        inside_borders = morphology.dilation(inside_borders, morphology.disk(20))
        convex_skeleton[inside_borders.astype(bool)] = 0
        # найдем компоненты связности
        labels, num = measure.label(convex_skeleton, return_num=True)
        # не будем учитывать случайный мусор - компоненты небольшого размера
        counter = 0
        for i in range(1, num + 2):
            if np.sum(labels == i) > 200:
                counter += 1
        if(counter == 4):
            print('Граф класса 2')
        elif(counter == 5):
            print('Граф класса 4')